## Naive Bayes

In [1]:
import pandas as pd
import pickle

In [2]:
data = pickle.load(open('data/df_clean_der.pkl','rb'))

In [3]:
inflation = data.loc[data.topics == 'inflation']
inflation.shape

(33, 7)

In [4]:
exchange = data.loc[data.topics == 'exchange']
exchange.shape

(28, 7)

In [5]:
corpus = inflation.append(exchange)
corpus.shape

(61, 7)

In [6]:
corpus

,topics,date,flyer,title,lead,body,url
0,inflation,29/11/2020,Economía,La enfermedad crónica que tiene Argentina: cas...,Desde la década de 1940 Argentina padece una e...,Argentina sirve como uno de los países más par...,https://derechadiario.com.ar/economia/la-enfer...
1,inflation,12/11/2020,Economía,Se acelera el aumento de precios: la inflación...,El Índice de Precios al Consumidor aumentó un ...,El INDEC divulgó las cifras oficiales para el ...,https://derechadiario.com.ar/economia/se-acele...
2,inflation,11/11/2020,Economía,Socialismo y miseria: la economía venezolana c...,El régimen del dictador Maduro continúa destru...,Desde la toma ilegal del poder de Maduro hace ...,https://derechadiario.com.ar/economia/socialis...
3,inflation,09/11/2020,Economía,“Efecto Lipovetzky”: los alquileres siguen aum...,"La iniciativa del PRO, formulada originalmente...",En un año marcado por las nuevas regulaciones ...,https://derechadiario.com.ar/economia/efecto-l...
4,inflation,02/11/2020,Economía,El Plan Guzmán para llegar a fin de año: más d...,Guzmán lanzó un quinto paquete de medidas orie...,"Esta mañana, el ministro Guzmán lanzó un inusu...",https://derechadiario.com.ar/economia/el-plan-...
...,...,...,...,...,...,...,...
56,exchange,17/04/2020,Economía,El dólar paralelo superó los $100 y ya no pare...,Alberto Fernández es el primero desde Alfonsín...,"“Son manionbras especulativas, al igual que es...",https://derechadiario.com.ar/economia/el-dolar...
57,exchange,14/04/2020,Economía,Se dispara el dólar: alcanzó los 114 pesos y c...,El tipo de cambio CCL escaló hasta la alarmant...,La jornada de hoy se caracterizó por una fuert...,https://derechadiario.com.ar/economia/se-dispa...
58,exchange,04/04/2020,Economía,Inminente levantamiento de las restricciones s...,El gobierno se prepara para levantar el parkin...,"Comisión Nacional de Valores (CNV), Esta decis...",https://derechadiario.com.ar/economia/inminent...
59,exchange,12/03/2020,Economía,Argentina en crisis: se dispara el riesgo país...,"El Riesgo País sigue sin encontrar techo, sube...",Una nueva\nmañana comenzó este jueves 12 de ma...,https://derechadiario.com.ar/economia/argentin...


#### Train - Test Split

In [7]:
X = corpus.body

In [8]:
y = corpus.topics

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train,test,y_train,y_test = train_test_split(X, y, train_size=0.75); 

In [11]:
train.shape, test.shape

((45,), (16,))

### Tokenization + Vectorization
#### CountVectorizer

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import re

In [13]:
stopwords_sp = stopwords.words('spanish')

In [14]:
def removal(text):
    text = re.sub(r'(\d|\$|\%|\+)', '', text.lower())
    return re.sub(r'\d+', '', text)

In [15]:
vectorizer = CountVectorizer(stop_words = stopwords_sp, lowercase = True, strip_accents='unicode', preprocessor=removal)

In [16]:
vectorizer.fit(list(re.sub(r'(\d|\$|\%|\+)', '', doc) for doc in corpus.body))

CountVectorizer(preprocessor=<function removal at 0x7fd72e1b7200>,
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents='unicode')

In [17]:
X_train = vectorizer.transform(train);

In [18]:
X_train

<45x4549 sparse matrix of type '<class 'numpy.int64'>'
	with 9461 stored elements in Compressed Sparse Row format>

In [19]:
X_train_matriz = X_train.todense()

In [20]:
X_test = vectorizer.transform(test);

In [21]:
X_test_matriz = X_train.todense()

#### Tf IDf Vectorizer

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer

In [23]:
X_train_matriz_tfidf = TfidfTransformer().fit_transform(X_train_matriz);

In [24]:
X_test_matriz_tfidf = TfidfTransformer().fit_transform(X_test_matriz);

### Multinomial Naive Bayes + CountVectorizer

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [26]:
multi_nb = MultinomialNB()

multi_nb.fit(X_train, y_train.values)

MultinomialNB()

In [27]:
y_pred = multi_nb.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
accuracy_score(y_pred, y_test.values)

0.9375

### Multinomial Naive Bayes + TfIDf

In [30]:
multi_nb_2 = MultinomialNB()

multi_nb_2.fit(X_train_matriz_tfidf, y_train.values)

MultinomialNB()

In [31]:
y_pred = multi_nb_2.predict(X_test)

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
accuracy_score(y_pred, y_test.values)

0.9375

### Tokenization + Vectorization (Manual)

#### CountVectorizer with pre-built tokens

In [34]:
cv_tokens = pickle.load(open('tokens/topic_tokens_der_new.pkl', 'rb'))

In [35]:
vectorizer.fit(cv_tokens.drop('topics', axis=1))

CountVectorizer(preprocessor=<function removal at 0x7fd72e1b7200>,
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents='unicode')

In [36]:
X_train_2 = vectorizer.transform(train)

In [37]:
X_train_matriz_2 = X_train_2.todense()

In [38]:
X_test_2 = vectorizer.transform(test)

In [39]:
X_test_matriz_2 = X_test_2.todense()

#### TF-IDF Vectorizer with pre-built tokens

In [40]:
X_train_matriz_tfidf_2 = TfidfTransformer().fit_transform(X_train_matriz_2);

In [41]:
X_test_matriz_tfidf_2 = TfidfTransformer().fit_transform(X_test_matriz_2);

### Multinomial Naive Bayes + CountVectorizer (updated tokens)

In [42]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [43]:
multi_nb = MultinomialNB()

multi_nb.fit(X_train_2, y_train.values)

MultinomialNB()

In [44]:
y_pred_2 = multi_nb.predict(X_test_2)

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
accuracy_score(y_pred_2, y_test.values)

0.9375

### Multinomial Naive Bayes + Tf IDf Vectorizer (updated tokens)

In [47]:
multi_nb_2 = MultinomialNB()

multi_nb_2.fit(X_train_matriz_tfidf_2, y_train.values)

MultinomialNB()

In [48]:
y_pred_2 = multi_nb_2.predict(X_test_matriz_tfidf_2)

In [49]:
from sklearn.metrics import accuracy_score

In [50]:
accuracy_score(y_pred_2, y_test.values)

0.9375

Multinomial Naive Bayes models finds more trouble to accuretly classify "derecha diario" news by topic. These scores can be explained from the results delivered by the SVD and PCA models, which proved that several news regarding inflation make use rather of a more monetary or financial lexicon to talk about the main topic.